In [1]:
from lmfdb import db
from lmfdb_tools import ec_growth_torsion
import pickle

INFO:LMFDB@2024-10-25 07:47:59,485: Configuration = {'flask_options': {'port': 37777, 'host': '127.0.0.1', 'debug': True, 'use_reloader': False}, 'postgresql_options': {'port': 5432, 'host': 'devmirror.lmfdb.xyz', 'dbname': 'lmfdb', 'user': 'lmfdb', 'password': '****'}, 'logging_options': {'logfile': 'flasklog', 'slowcutoff': 0.1, 'slowlogfile': 'slow_queries.log', 'editor': '', 'loglevel': 20}} 
INFO:LMFDB@2024-10-25 07:47:59,646: Connecting to PostgresSQL server as: user=lmfdb host=devmirror.lmfdb.xyz port=5432 dbname=lmfdb... 
INFO:LMFDB@2024-10-25 07:48:02,150: Done!
 connection = <connection object at 0x7630bcc1a200; dsn: 'user=lmfdb password=xxx dbname=lmfdb host=devmirror.lmfdb.xyz port=5432', closed: 0> 
2024-10-25 07:48:04,203 - SELECT pg_is_in_recovery() ran in  0.3170769214630127s 
2024-10-25 07:48:05,247 - SELECT current_setting('is_superuser') ran in  0.6472642421722412s 
INFO:LMFDB@2024-10-25 07:48:05,726: User: lmfdb 
INFO:LMFDB@2024-10-25 07:48:05,728: Read only: True 


# Descarga de todas las curvas con crecimiento de la torsión sobre algún cuerpo cuadrático

In [2]:
table = db.ec_torsion_growth.join_search(query={'degree' : int(2)}, 
                                        projection=["lmfdb_label", "torsion", "degree", "field", "conductor", ("ec_curvedata", "torsion_structure"),("ec_curvedata", "absD")], 
                                        join=[("ec_torsion_growth", "lmfdb_label", "ec_curvedata","lmfdb_label")])


ecs = list(table)
len(ecs)

2024-09-26 17:04:10,814 - SELECT "ec_torsion_growth"."lmfdb_label", "ec_torsion_growth"."torsion", "ec_torsion_growth"."degree", "ec_torsion_growth"."field", "ec_torsion_growth"."conductor", "ec_curvedata"."torsion_structure", "ec_curvedata"."absD" FROM "ec_torsion_growth" JOIN "ec_curvedata" ON "ec_torsion_growth"."lmfdb_label" = "ec_curvedata"."lmfdb_label" WHERE "ec_torsion_growth"."degree" = 2 ORDER BY "ec_torsion_growth"."lmfdb_label", "ec_torsion_growth"."degree" ran in  0.8296816349029541s 
2024-09-26 17:04:10,816 - Replicate with db.ec_torsion_growth.analyze({'degree': 2}, ['lmfdb_label', 'torsion', 'degree', 'field', 'conductor', ('ec_curvedata', 'torsion_structure'), ('ec_curvedata', 'absD')], None, 0)
2024-09-26 17:05:23,012 - Search iterator for ec_torsion_growth {'degree': 2} required a total of 70.61548805236816s


KeyboardInterrupt: 

In [ ]:
x = polygen(ZZ,'x')

for ec in ecs:
    ec['torsion_before'] = ec[('ec_curvedata','torsion_structure')]
    ec['torsion_after'] = ec['torsion']
    ec['curve_discriminant'] = ec[('ec_curvedata','absD')]
    
    del ec['torsion']
    del ec[('ec_curvedata','torsion_structure')]
    del ec[('ec_curvedata','absD')]
    
    f = ec['field']
    K.<alpha> = NumberField(f[0]*x^2+ f[1]*x + f[2]) 
    ec['disc_abs'] = K.absolute_discriminant()

In [1]:
import pickle

with open('torsion_growth.pkl', 'wb') as file:
    pickle.dump(ecs, file)


NameError: name 'ecs' is not defined

# Funciones auxiliares

In [5]:
def prod(xs):
    prod = 1
    for x in xs:
        prod *= x
    return prod

def conditions(curve,  dict_key_property):
    boolean = True
    for key, propertyy in  dict_key_property.items():
        boolean = boolean and propertyy(curve[key])
    return boolean

def growth_torsion_search(dict_possible_torsion_growth, curves_list):
    return [ec for ec in curves_list if (tuple(ec['torsion_before']) in dict_possible_torsion_growth.keys()) 
            and (ec['torsion_after'] in dict_possible_torsion_growth[tuple(ec['torsion_before'])])]
    
    

In [5]:
dict_conditions = {'conductor' : lambda cond : cond%3 != 0, 
                   'torsion_before' : lambda tor_list : prod(tor_list)% 3 != 0,
                   'torsion_after' : lambda tor_list: prod(tor_list)%3 == 0,
                   'disc_abs' : lambda disc: disc%3 == 0}

ecs_trivial2C3 = [ec for ec in ecs if  conditions(ec, dict_conditions)]

In [6]:
counterexamples = [ec for ec in ecs_trivial2C3 if prod(ec['torsion_after'])/prod(ec['torsion_before']) != 3]
len(counterexamples)

0

# Mejora de las conjeturas anteriores (reducción aditiva)

In [6]:
query = [('ec_torsion_growth', {'degree' : int(2)}), ('ec_localdata',{'reduction_type' : int(0)})]
table = db.ec_torsion_growth.join_search(query=query, 
                                        projection=["lmfdb_label", "torsion", "degree", "field", "conductor", ("ec_curvedata", "torsion_structure"),("ec_curvedata", "absD"),("ec_localdata", "reduction_type"),("ec_localdata","prime")], 
                                        join=[("ec_torsion_growth", "lmfdb_label", "ec_curvedata","lmfdb_label"), ("ec_torsion_growth", "lmfdb_label", "ec_localdata","lmfdb_label")])

ecs_torsion_growth_additive_reduction_raw = list(table)
len(ecs_torsion_growth_additive_reduction)

2024-06-25 21:08:03,437 - SELECT "ec_torsion_growth"."lmfdb_label", "ec_torsion_growth"."torsion", "ec_torsion_growth"."degree", "ec_torsion_growth"."field", "ec_torsion_growth"."conductor", "ec_curvedata"."torsion_structure", "ec_curvedata"."absD", "ec_localdata"."reduction_type", "ec_localdata"."prime" FROM "ec_torsion_growth" JOIN "ec_curvedata" ON "ec_torsion_growth"."lmfdb_label" = "ec_curvedata"."lmfdb_label" JOIN "ec_localdata" ON "ec_torsion_growth"."lmfdb_label" = "ec_localdata"."lmfdb_label" WHERE "ec_torsion_growth"."degree" = 2AND"ec_localdata"."reduction_type" = 0 ORDER BY "ec_torsion_growth"."lmfdb_label", "ec_torsion_growth"."degree" ran in  0.24469423294067383s 
2024-06-25 21:08:03,439 - Replicate with db.ec_torsion_growth.analyze([('ec_torsion_growth', {'degree': 2}), ('ec_localdata', {'reduction_type': 0})], ['lmfdb_label', 'torsion', 'degree', 'field', 'conductor', ('ec_curvedata', 'torsion_structure'), ('ec_curvedata', 'absD'), ('ec_localdata', 'reduction_type'), ('

1894605

In [13]:
x = polygen(ZZ,'x')

for ec in ecs_torsion_growth_additive_reduction_raw: 
    ec['torsion_before'] = ec[('ec_curvedata','torsion_structure')]
    ec['torsion_after'] = ec['torsion']
    ec['curve_discriminant'] = ec[('ec_curvedata','absD')]
    
    del ec['torsion']
    del ec[('ec_curvedata','torsion_structure')]
    del ec[('ec_curvedata','absD')]
    
    f = ec['field']
    K.<alpha> = NumberField(f[0]*x^2+ f[1]*x + f[2]) 
    ec['disc_abs'] = K.absolute_discriminant()

In [18]:
badprimes_dict = dict()
for ec in ecs_torsion_growth_additive_reduction_raw:
    badprimes_dict[ec['lmfdb_label']] = set()

In [19]:
for ec in ecs_torsion_growth_additive_reduction_raw:
    badprimes_dict[ec['lmfdb_label']].add(ec[('ec_localdata', 'prime')])

In [21]:
for ec in ecs_torsion_growth_additive_reduction_raw:
    ec['torsion_before'] = ec[('ec_curvedata','torsion_structure')]
    ec['torsion_after'] = ec['torsion']
    ec['curve_discriminant'] = ec[('ec_curvedata','absD')]
    ec['additive_reduction_primes'] = badprimes_dict[ec['lmfdb_label']]
    
    del ec['torsion']
    del ec[('ec_curvedata','torsion_structure')]
    del ec[('ec_curvedata','absD')]
    del ec[('ec_localdata','reduction_type')]
    del ec[('ec_localdata','prime')]
    
    f = ec['field']
    K.<alpha> = NumberField(f[0]*x^2+ f[1]*x + f[2]) 
    ec['disc_abs_field'] = K.absolute_discriminant()

In [22]:
ecs_torsion_growth_additive_reduction= ecs_torsion_growth_additive_reduction_raw

In [23]:
ecs_torsion_growth_additive_reduction[0]

{'lmfdb_label': '100.a1',
 'degree': 2,
 'field': [-1, -1, 1],
 'conductor': 100,
 'torsion_before': [2],
 'torsion_after': [2, 2],
 'curve_discriminant': 31250000,
 'additive_reduction_primes': {2, 5},
 'disc_abs_field': 5}

### Conjeturas mejoradas

In [33]:
torsion_3 = growth_torsion_search({():[[3]]},ecs_torsion_growth_additive_reduction)
torsion_5 = growth_torsion_search({() : [[5]], (2,) : [[10]]}, ecs_torsion_growth_additive_reduction)
torsion_7 = growth_torsion_search({() : [[7]]}, ecs_torsion_growth_additive_reduction)
torsion_9 = growth_torsion_search({():[[9]]},ecs_torsion_growth_additive_reduction)


In [39]:
def conjecture_point(order, curves_list):
    bad_curves = []
    for ec in curves_list:
        if not all([ ec['disc_abs_field']%p == 0 for p in ec['additive_reduction_primes'] if p!=order]):
            bad_curves.append(ec)
    return bad_curves

In [47]:
conjecture_point(1,torsion_5)

[{'lmfdb_label': '100450.cg2',
  'degree': 2,
  'field': [2, -1, 1],
  'conductor': 100450,
  'torsion_before': [],
  'torsion_after': [5],
  'curve_discriminant': 2125211801039667200,
  'additive_reduction_primes': {5, 7},
  'disc_abs_field': -7},
 {'lmfdb_label': '100450.cg2',
  'degree': 2,
  'field': [2, -1, 1],
  'conductor': 100450,
  'torsion_before': [],
  'torsion_after': [5],
  'curve_discriminant': 2125211801039667200,
  'additive_reduction_primes': {5, 7},
  'disc_abs_field': -7},
 {'lmfdb_label': '100800.cj2',
  'degree': 2,
  'field': [-6, 0, 1],
  'conductor': 100800,
  'torsion_before': [],
  'torsion_after': [5],
  'curve_discriminant': 624387084307660800,
  'additive_reduction_primes': {2, 3, 5},
  'disc_abs_field': 24},
 {'lmfdb_label': '100800.cj2',
  'degree': 2,
  'field': [-6, 0, 1],
  'conductor': 100800,
  'torsion_before': [],
  'torsion_after': [5],
  'curve_discriminant': 624387084307660800,
  'additive_reduction_primes': {2, 3, 5},
  'disc_abs_field': 24},


### Quitamos las curvas que ponen un $\mathcal{C}_2$ y ya tenían un $\mathcal{C}_2$

In [13]:
def removeC2_prop(ec):
    return (len(ec['torsion_before']) == 1) and (ec['torsion_before'][0] % 2 == 0) and (len(ec['torsion_after']) == 2) and (ec['torsion_after'][1]%2 == 0)

curves_torsion = [ec for ec in curves_w_prop if not(removeC2_prop(ec))]
len(curves_torsion)

531895

{'lmfdb_label': '100016.k3',
 'degree': 2,
 'field': [-19, 0, 1],
 'conductor': 100016,
 'torsion_before': [2],
 'torsion_after': [4],
 'disc_abs': 76}

In [14]:
C2_C4_curves = [ec for ec in curves_torsion if ec['torsion_before'] == [2] and ec['torsion_after'] ==[4]]

In [15]:
len(C2_C4_curves)

338674

In [55]:
def power_bigger_property(ec,bound):
    n = ec['curve_discriminant']
    return any([b== bound and (ec['disc_abs']%a == 0)for a,b in list(factor(n)) if not (a in [2])])


curves_big_exponents = [ec for ec in C2_C4_curves if power_bigger_property(ec,)]
len(curves_big_exponents)

0

In [30]:
curves_big_exponents[1]

{'lmfdb_label': '100555.q3',
 'degree': 2,
 'field': [-3, -1, 1],
 'conductor': 100555,
 'torsion_before': [2],
 'torsion_after': [4],
 'curve_discriminant': 71772577318885625,
 'disc_abs': 13}

[([2], [2, 2]),
 ([2, 2], [2, 4]),
 ([2], [4]),
 ([4], [2, 4]),
 ([], [3]),
 ([6], [2, 6]),
 ([4], [8]),
 ([2], [6]),
 ([2, 4], [2, 8]),
 ([], [5]),
 ([], [7]),
 ([2], [8]),
 ([2, 2], [2, 8]),
 ([8], [2, 8]),
 ([10], [2, 10]),
 ([2], [2, 6]),
 ([2, 2], [2, 6]),
 ([2], [12]),
 ([2], [10]),
 ([6], [12]),
 ([8], [16]),
 ([4], [12]),
 ([2], [2, 10]),
 ([4], [2, 12]),
 ([12], [2, 12]),
 ([], [9]),
 ([4], [2, 8]),
 ([2, 6], [2, 12]),
 ([2, 2], [2, 12])]

# Conjeturas sobre el problema inverso de la torsión $\mathcal{C}_1\rightarrow {\mathcal{C}_3,\mathcal{C}_5,\mathcal{C}_7,\mathcal{C}_9}$

In [2]:
with open('torsion_growth_quadratic.pkl', 'rb') as file:
    loaded_list = pickle.load(file)

In [3]:
len(loaded_list)

1661447

In [6]:
torsion_7 = growth_torsion_search({() : [[7]]}, loaded_list)
torsion_5 = growth_torsion_search({() : [[5]]}, loaded_list)
torsion_9 = growth_torsion_search({():[[9]]},loaded_list)
torsion_3 = growth_torsion_search({():[[3]]},loaded_list)

In [12]:
[ec for ec in torsion_9 if ec['conductor']%3 !=0]

[]

In [85]:
conjecture1(torsion_7)

[{'lmfdb_label': '100672.ee2',
  'degree': 2,
  'field': [22, 0, 1],
  'conductor': 100672,
  'torsion_before': [],
  'torsion_after': [7],
  'curve_discriminant': 772768400408576,
  'disc_abs': -88},
 {'lmfdb_label': '100672.f2',
  'degree': 2,
  'field': [-22, 0, 1],
  'conductor': 100672,
  'torsion_before': [],
  'torsion_after': [7],
  'curve_discriminant': 772768400408576,
  'disc_abs': 88},
 {'lmfdb_label': '101136.cn2',
  'degree': 2,
  'field': [-7, 0, 1],
  'conductor': 101136,
  'torsion_before': [],
  'torsion_after': [7],
  'curve_discriminant': 742481036549554176,
  'disc_abs': 28},
 {'lmfdb_label': '10192.a2',
  'degree': 2,
  'field': [-7, 0, 1],
  'conductor': 10192,
  'torsion_before': [],
  'torsion_after': [7],
  'curve_discriminant': 801865465856,
  'disc_abs': 28},
 {'lmfdb_label': '104400.dy2',
  'degree': 2,
  'field': [-15, 0, 1],
  'conductor': 104400,
  'torsion_before': [],
  'torsion_after': [7],
  'curve_discriminant': 378760126464000000,
  'disc_abs': 60}

In [87]:
def conjecture1(curves_list):
    bad_curves = []
    for ec in curves_list:
        factorization = list(factor(ec['conductor']))
        if not all([(p==7 or p ==2)  or ((e==2) and (ec['disc_abs']%p == 0)) or ((e==1) and (ec['disc_abs']%p != 0))  for p,e in factorization]):
            bad_curves.append(ec)
    return bad_curves
        
        
def conjecture2(curves_list):
    bad_curves = []
    for ec in curves_list:
        factorization = list(factor(ec['conductor']))
        if not all([(p==2 or p == 5)  or ((e==2) and (ec['disc_abs']%p == 0)) or ((e==1) and (ec['disc_abs']%p != 0))  for p,e in factorization]):
            bad_curves.append(ec)
    return bad_curves
        
    
def conjecture3(curves_list):
    bad_curves = []
    for ec in curves_list:
        factorization = list(factor(ec['conductor']))
        if not all([(p==2 or p == 3)  or ((e==2) and (ec['disc_abs']%p == 0)) or ((e==1) and (ec['disc_abs']%p != 0))  for p,e in factorization]):
            bad_curves.append(ec)
    return bad_curves      

In [88]:
def generalized_conjecture1(curves_list):
    bad_curves = []
    for ec in curves_list:
        factorization_disc = list(factor(ec['disc_abs']))
        factorization_cond = list(factor(ec['conductor']))
        conj_1 = all([(p==2 or p==7)  or ((e==2) and (ec['disc_abs']%p == 0)) or ((e==1) and (ec['disc_abs']%p != 0))  for p,e in factorization_cond])
        aux = all([(ec['conductor']%p^2 == 0) for p,e in factorization_disc]) 
        if  not(conj_1 and aux):
            bad_curves.append(ec)
    return bad_curves

def generalized_conjecture2(curves_list):
    bad_curves = []
    for ec in curves_list:
        factorization_disc = list(factor(ec['disc_abs']))
        factorization_cond = list(factor(ec['conductor']))
        conj_1 = all([(p==2 or p==5)  or ((e==2) and (ec['disc_abs']%p == 0)) or ((e==1) and (ec['disc_abs']%p != 0))  for p,e in factorization_cond])
        aux = all([(ec['conductor']%p^2 == 0) for p,e in factorization_disc]) 
        if  not(conj_1 and aux):
            bad_curves.append(ec)
    return bad_curves

def generalized_conjecture3(curves_list):
    bad_curves = []
    for ec in curves_list:
        factorization_disc = list(factor(ec['disc_abs']))
        factorization_cond = list(factor(ec['conductor']))
        conj_1 = all([(p==2 or p==3)  or ((e==2) and (ec['disc_abs']%p == 0)) or ((e==1) and (ec['disc_abs']%p != 0))  for p,e in factorization_cond])
        aux = all([(ec['conductor']%p^2 == 0) for p,e in factorization_disc]) 
        if  not(conj_1 and aux):
            bad_curves.append(ec)
    return bad_curves


def generalized_conjecture4(curves_list):
    bad_curves = []
    for ec in curves_list:
        factorization_disc = list(factor(ec['disc_abs']))
        factorization_cond = list(factor(ec['conductor']))
        conj_1 = all([(p==2 or p==3)  or ((e==2) and (ec['disc_abs']%p == 0)) or ((e==1) and (ec['disc_abs']%p != 0))  for p,e in factorization_cond])
        aux = all([(ec['conductor']%p^2 == 0) for p,e in factorization_disc]) 
        if  not aux:
            bad_curves.append(ec)
    return bad_curves

In [73]:
conjecture3(torsion_3)[0]

{'lmfdb_label': '100254.ck1',
 'degree': 2,
 'field': [1, -1, 1],
 'conductor': 100254,
 'torsion_before': [],
 'torsion_after': [3],
 'curve_discriminant': 592966070227718151519504187392,
 'disc_abs': -3}

# Conjeturas sobre el problema inverso de la torsión $\mathcal{C}_2\rightarrow \mathcal{C}_{10}$

In [ ]:
with open('torsion_growth_quadratic.pkl', 'rb') as file:
    loaded_list = pickle.load(file)

In [90]:
torsion2to10 = growth_torsion_search({(2,):[[10]]},loaded_list)
len(torsion2to10)

520

In [92]:
generalized_conjecture2(torsion2to10)

[]

In [77]:
factor(329)

7 * 47

# Petición de Enrique

In [1]:
from lmfdb import db
from lmfdb_tools import ec_growth_torsion
import pickle

INFO:LMFDB@2024-06-14 09:31:57,049: Configuration = {'flask_options': {'port': 37777, 'host': '127.0.0.1', 'debug': True, 'use_reloader': False}, 'postgresql_options': {'port': 5432, 'host': 'devmirror.lmfdb.xyz', 'dbname': 'lmfdb', 'user': 'lmfdb', 'password': '****'}, 'logging_options': {'logfile': 'flasklog', 'slowcutoff': 0.1, 'slowlogfile': 'slow_queries.log', 'editor': '', 'loglevel': 20}} 
INFO:LMFDB@2024-06-14 09:31:57,215: Connecting to PostgresSQL server as: user=lmfdb host=devmirror.lmfdb.xyz port=5432 dbname=lmfdb... 
INFO:LMFDB@2024-06-14 09:31:58,548: Done!
 connection = <connection object at 0x781a62b3e200; dsn: 'user=lmfdb password=xxx dbname=lmfdb host=devmirror.lmfdb.xyz port=5432', closed: 0> 
2024-06-14 09:31:59,571 - SELECT pg_is_in_recovery() ran in  0.20370078086853027s 
2024-06-14 09:32:00,050 - SELECT current_setting('is_superuser') ran in  0.27348756790161133s 
INFO:LMFDB@2024-06-14 09:32:00,175: User: lmfdb 
INFO:LMFDB@2024-06-14 09:32:00,177: Read only: True

In [15]:
table = db.ec_torsion_growth.join_search(query={'degree' : int(2), 'torsion': [3]} , 
                                        projection=["lmfdb_label", "torsion", "degree", "field", "conductor", ("ec_curvedata", "torsion_structure"),("ec_curvedata", "absD"),("ec_curvedata",'modell_images')], 
                                        join=[("ec_torsion_growth", "lmfdb_label", "ec_curvedata","lmfdb_label")])



2024-06-14 10:11:54,999 - SELECT "ec_torsion_growth"."lmfdb_label", "ec_torsion_growth"."torsion", "ec_torsion_growth"."degree", "ec_torsion_growth"."field", "ec_torsion_growth"."conductor", "ec_curvedata"."torsion_structure", "ec_curvedata"."absD", "ec_curvedata"."modell_images" FROM "ec_torsion_growth" JOIN "ec_curvedata" ON "ec_torsion_growth"."lmfdb_label" = "ec_curvedata"."lmfdb_label" WHERE "ec_torsion_growth"."degree" = 2 AND "ec_torsion_growth"."torsion" = ARRAY[3]::smallint[] ORDER BY "ec_torsion_growth"."lmfdb_label", "ec_torsion_growth"."degree" ran in  0.5095007419586182s 
2024-06-14 10:11:55,001 - Replicate with db.ec_torsion_growth.analyze({'degree': 2, 'torsion': [3]}, ['lmfdb_label', 'torsion', 'degree', 'field', 'conductor', ('ec_curvedata', 'torsion_structure'), ('ec_curvedata', 'absD'), ('ec_curvedata', 'modell_images')], None, 0)


In [16]:
enrique_curves = list(table)

2024-06-14 10:13:16,819 - Search iterator for ec_torsion_growth {'degree': 2, 'torsion': [3]} required a total of 79.5492799282074s


In [17]:
len(enrique_curves)

141560

In [18]:
enrique_curves[0]

{'lmfdb_label': '100014.e1',
 'torsion': [3],
 'degree': 2,
 'field': [1, -1, 1],
 'conductor': 100014,
 ('ec_curvedata', 'torsion_structure'): [],
 ('ec_curvedata', 'absD'): 2250945132306174,
 ('ec_curvedata', 'modell_images'): ['3B.1.2']}

In [19]:
for ec in enrique_curves:
    ec['torsion_before'] = ec[('ec_curvedata','torsion_structure')]
    ec['torsion_after'] = ec['torsion']
    ec['curve_discriminant'] = ec[('ec_curvedata','absD')]
    ec['modell_images'] = ec[('ec_curvedata','modell_images')] 
    
    del ec['torsion']
    del ec[('ec_curvedata','torsion_structure')]
    del ec[('ec_curvedata','absD')]
    del ec[('ec_curvedata','modell_images')]

In [28]:
for ec in enrique_curves:
    del ec['degree']

In [23]:
import pickle

In [29]:
with open('enrique_curves_1.pkl', 'wb') as file:
    pickle.dump(enrique_curves, file)


In [30]:
import pickle

with open('enrique_curves_1.pkl', 'rb') as file:
    loaded_list = pickle.load(file)



In [31]:
loaded_list[0]

{'lmfdb_label': '100014.e1',
 'field': [1, -1, 1],
 'conductor': 100014,
 'torsion_before': [],
 'torsion_after': [3],
 'curve_discriminant': 2250945132306174,
 'modell_images': ['3B.1.2']}

In [36]:
images = set()
for ec in loaded_list:
    for image in ec['modell_images']:
        images.add(image)

In [37]:
images

{'101Nn.1.31',
 '103Ns.3.1.2',
 '113Nn.1.36',
 '11Nn.1.4',
 '131Nn.1.40',
 '137Nn.1.38',
 '139Ns.8.1.3',
 '13Ns.5.1.4',
 '149Nn.1.44',
 '151Ns.3.1.4',
 '157Ns.2.1.9',
 '167Nn.1.14',
 '173Nn.1.18',
 '191Nn.2.58',
 '193Ns.11.1.4',
 '23Nn.1.11',
 '29Nn.1.14',
 '2Cn',
 '31Ns.15.1.5',
 '3B',
 '3B.1.2',
 '3Cs',
 '41Nn.2.16',
 '43Ns.2.1.7',
 '47Nn.1.19',
 '59Nn.1.26',
 '5B.1.2',
 '5B.1.3',
 '5B.1.4',
 '5B.4.1',
 '5B.4.2',
 '5Ns.2.1',
 '5S4',
 '61Ns.8.1.4',
 '67Ns.3.1.4',
 '79Ns.12.1.4',
 '7B',
 '7B.2.1',
 '7B.2.3',
 '7Ns.6.1.2',
 '83Nn.1.31',
 '97Ns.19.1.4'}

KeyboardInterrupt: 

In [19]:
x = polygen(ZZ,'x')
b = x^5-2*x^4 + 2*x^3-x^2
c = x^3-x^2
delta = (1-c)^4*b^3- (1-c)^3*b^3- 8*(1-c)^2*b^4+36*(1-c)*b^4-27*b^4+16*b^5

In [21]:
delta.factor()

(x - 1)^9 * x^9 * (x^2 - x + 1)^3 * (x^3 - 6*x^2 + 3*x + 1)

In [22]:
b.factor()

(x - 1) * x^2 * (x^2 - x + 1)

In [23]:
c.factor()

(x - 1) * x^2

In [3]:
import pickle

with open('torsion_growth_quadratic.pkl', 'rb') as file:
    loaded_list = pickle.load(file)


In [4]:
loaded_list[0]

{'lmfdb_label': '100.a1',
 'degree': 2,
 'field': [-1, -1, 1],
 'conductor': 100,
 'torsion_before': [2],
 'torsion_after': [2, 2],
 'curve_discriminant': 31250000,
 'disc_abs': 5}

In [8]:
C2_C2C6_curves = [ec for ec in loaded_list if ec['torsion_before'] == [2] and ec['torsion_after'] ==[2,6]]

In [12]:
C2_C2C6_curves[2]

{'lmfdb_label': '100800.mh2',
 'degree': 2,
 'field': [-30, 0, 1],
 'conductor': 100800,
 'torsion_before': [2],
 'torsion_after': [2, 6],
 'curve_discriminant': 7742895390720000000,
 'disc_abs': 120}